<a href="https://colab.research.google.com/github/wyasul/cs4701_Cornbot/blob/main/Cornbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing gensim (if needed)
!pip install -U gensim
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 36.0 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.6 MB/s 
     |████████████████████████████████| 5.8 MB 11.3 MB/s 
     |████████████████████████████████| 1.3 MB 56.6 MB/s 
     |████████████████████████████████| 182 kB 49.5 MB/s 
     |████████████████████████████████| 7.6 MB 24.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7f4495af0aca80e3ddec7491898f891b7e3b81129e25b4f0fac489b6aacfb8ad
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully bu

In [ ]:
from collections import Counter, namedtuple
from itertools import chain
import json
import math
import os
from pathlib import Path
from tqdm.notebook import tqdm, trange
from typing import List, Tuple, Dict, Set, Union
import matplotlib.pyplot as plt

import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.utils
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### TODO : please modify the line below with your drive organization ###
path = os.path.join(os.getcwd(), "drive", "MyDrive")

# with open(os.path.join(path,'cornbot_train.json'), 'r') as f:
#      train = json.loads(f.read())
with open(os.path.join(path,'cornbot_train.json'), 'r') as f:
     train = json.loads(f.read())

with open(os.path.join(path,'cornbot_validation.json'), 'r') as f:
     val = json.loads(f.read())

with open(os.path.join(path,'cornbot_test.json'), 'r') as f:
      test = json.loads(f.read())

Mounted at /content/drive


In [ ]:
text = []
text_str = []
text_untokenized = []
tags = []
flag = False
for intent in train["intents"]:
    for pattern in intent["patterns"]:
        text.append(pattern.split())
        for word in pattern.split():
            text_str.append(word)
        tags.append(intent["tag"])
        text_untokenized.append(pattern)

val_text = []
val_text_untokenized = []
val_tags = []
for intent in val["intents"]:
    for pattern in intent["patterns"]:
        val_text.append(pattern.split())
        val_tags.append(intent["tag"])
        val_text_untokenized.append(pattern)

test_text = []
test_text_untokenized = []
test_tags = []
for intent in test["intents"]:
    for pattern in intent["patterns"]:
        test_text.append(pattern.split())
        test_tags.append(intent["tag"])
        test_text_untokenized.append(pattern)
print(test_text_untokenized)

['Fall add deadline', "When's fall sem add deadline?", 'When is the last day to add a course this fall?', 'Fall drop deadline?', "What's the fall drop deadline?", 'When is the last day to drop a course this fall?', 'Spring add deadline', "When's spring sem add deadline?", 'When is the last day to add a course this spring?', 'Spring drop deadline?', "What's the spring drop deadline?", 'When is the last day to drop a course this spring?', "When's fall break?", 'When does fall break start?', 'Fall break', "When's winter break?", 'When does winter break start?', 'Winter break', "When's thanksgiving break?", 'When does thanksgiving break start?', 'Thanksgiving break', "When's spring break?", 'When does spring break start?', 'Spring break', "When's february break?", 'When does february break start?', 'February break', "When's last day of spring?", 'Last day of spring', 'How long until last day of spring classes?', "When's last day of fall?", 'Last day of fall', 'How long until last day of fa

In [ ]:
UNK = "<UNK>"
PAD = "<PAD>"

class Vocab(object):
    def __init__(self, word2id=None):
        if word2id:
            self.word2id = word2id
        else:
            self.word2id = dict()
            self.word2id[PAD] = 0   # Pad Token
            self.word2id[UNK] = 1   # Unknown Token
        self.unk_id = self.word2id[UNK]
        self.id2word = {v: k for k, v in self.word2id.items()}

    def __getitem__(self, word):
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        return word in self.word2id

    def __setitem__(self, key, value):
        raise ValueError('vocabulary is readonly')

    def __len__(self):
        return len(self.word2id)

    def __repr__(self):
        return 'Vocabulary[size=%d]' % len(self)

    def word_from_id(self, wid):
        return self.id2word[wid]

    def add(self, word):
        if word not in self:
            wid = self.word2id[word] = len(self)
            self.id2word[wid] = word
            return wid
        else:
            return self[word]

    @staticmethod
    def from_corpus(corpus): 
        vocab_entry = Vocab()
        top_words = set(corpus)
        for word in top_words:
            vocab_entry.add(word.translate(str.maketrans('', '', '.?,:;!')))
        return vocab_entry

In [ ]:
def tokenize_sentences(sentences, vocab):
    tokenized_sents = []
    for sentence in sentences:
        tokenized_sentence = [vocab[w] for w in sentence]
        tokenized_sents.append(tokenized_sentence)

    output = []

    for i in range(len(tokenized_sents)):
        output.append([float(0) for v in range(vocab.__len__())])
        for j in range(len(tokenized_sents[i])):
            output[i][tokenized_sents[i][j]] = float(1)

    return output

In [ ]:
from sentence_transformers import SentenceTransformer
def bert_sentences(sentences):
    bert_model = SentenceTransformer('all-MiniLM-L6-v2')
    return bert_model.encode(sentences)

In [ ]:
max_len = 0

for sentence in text:
  if len(sentence) > max_len:
    max_len = len(sentence)

def pad_sents(sents, pad_token, max_len):
    for sent in sents:
      for i in range(max_len - len(sent)):
        sent.append(pad_token)

    return sents

In [ ]:
category_map = {}
counter = 0
for tag in tags:
    if tag not in category_map:
        category_map[tag] = counter
        counter += 1

print(category_map)

def encode_tag(tag_data):
  label_ints = []

  for tag in tag_data:
    label_ints.append(category_map[tag])

  return label_ints

{'add fall': 0, 'drop fall': 1, 'add spring': 2, 'drop spring': 3, 'fall break': 4, 'winter break': 5, 'thanksgiving break': 6, 'spring break': 7, 'february break': 8, 'last day spring': 9, 'last day fall': 10, 'fall exam period': 11, 'fall study period': 12, 'december degree date': 13, 'spring exam period': 14, 'spring study period': 15, 'winter session': 16, 'summer session': 17, 'pre-enroll freshmen fall': 18, 'pre-enroll sophomore fall': 19, 'pre-enroll junior fall': 20, 'pre-enroll senior fall': 21, 'dining hall hours 104west': 22, 'dining hall hours becker': 23, 'dining hall hours cook': 24, 'dining hall hours bethe': 25, 'dining hall hours keeton': 26, 'dining hall hours morrison': 27, 'dining hall hours okenshields': 28, 'dining hall hours risley': 29, 'dining hall hours rose': 30, 'Macs': 31, 'Terrace': 32, 'Trillium': 33, 'chimes': 34, 'building location comstock': 35, 'building location goldwin smith': 36, 'building location cornell health': 37, 'building location hollister'

In [ ]:
torch.manual_seed(123)
print('initialize train vocabulary ..')
vocab = Vocab.from_corpus(text_str)

initialize train vocabulary ..


In [ ]:
from zmq.constants import PROTOCOL_ERROR_ZMTP_INVALID_SEQUENCE

processed_train = dict()
processed_train['tags'] = encode_tag(tags)
processed_train['text'] = pad_sents(text, PAD, max_len)
processed_train['text-tokenized'] = tokenize_sentences(processed_train['text'], vocab)

processed_val = dict()
processed_val['tags'] = encode_tag(val_tags)
processed_val['text'] = pad_sents(val_text, PAD, max_len)
processed_val['text-tokenized'] = tokenize_sentences(processed_val['text'], vocab)

processed_train_bert = dict()
processed_train_bert['tags'] = processed_train['tags']
processed_train_bert['text-tokenized'] = bert_sentences(text_untokenized)

processed_val_bert = dict()
processed_val_bert['tags'] = processed_val['tags']
processed_val_bert['text-tokenized'] = bert_sentences(val_text_untokenized)

processed_test = dict()
processed_test['tags'] = encode_tag(test_tags)
processed_test['text'] = pad_sents(test_text, PAD, max_len)
processed_test['text-tokenized'] = tokenize_sentences(processed_test['text'], vocab)

processed_test_bert = dict()
processed_test_bert['tags'] = processed_test['tags']
processed_test_bert['text-tokenized'] = bert_sentences(test_text_untokenized)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
class LanguageDataset(Dataset):
    def __init__(self, text, tag):
        self.text = torch.tensor(text)
        self.tag = torch.tensor(tag)
        self.len = len(text)
    
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        if not self.tag.numel():
             return self.text[index], []
        else:
            return self.text[index], self.tag[index]


In [ ]:
def get_data_loaders(preprocessed_data, batch_size=1, shuffle=False):
    dataset = LanguageDataset(preprocessed_data['text-tokenized'], preprocessed_data['tags'])
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

In [ ]:
torch.manual_seed(123)

batch_size = 200
train_loader = get_data_loaders(processed_train, batch_size=batch_size)
val_loader = get_data_loaders(processed_val, batch_size=batch_size)
train_loader_bert = get_data_loaders(processed_train_bert, batch_size=batch_size)
val_loader_bert = get_data_loaders(processed_val_bert, batch_size=batch_size)
test_loader = get_data_loaders(processed_test, batch_size=batch_size)
test_loader_bert = get_data_loaders(processed_test_bert, batch_size=batch_size)

In [ ]:
# Lambda to switch to GPU if available
get_device = lambda : "cuda:0" if torch.cuda.is_available() else "cpu"

# Setting seed ***DO NOT MODIFY***
torch.manual_seed(123)

class FFNN(nn.Module):
	def __init__(self, input_dim, hidden_dim, output_dim, vocab_size, nhidden = 1):
		super(FFNN, self).__init__()
		self.W1 = torch.nn.Linear(input_dim, hidden_dim)
		self.W2 = torch.nn.Linear(hidden_dim, hidden_dim)
		self.W3 = torch.nn.Linear(hidden_dim, output_dim)
		self.softmax = torch.nn.LogSoftmax(-1)
		self.loss = torch.nn.CrossEntropyLoss()
		self.nhidden = nhidden
  
	def compute_Loss(self, predicted_vector, gold_label):
		return self.loss(predicted_vector, gold_label)

	def forward(self, input_vector):
		z1 = self.W1(input_vector)
		z2 = self.W2(z1)
		for i in range(self.nhidden - 1):
			z2 = self.W2(z2)
		z3 = self.W3(z2)
		predicted_vector = self.softmax(z3)
		return predicted_vector

	def load_model(self, save_path, is_state_dict=False):
		if not is_state_dict:
			saved_model = torch.load(save_path)
			self.load_state_dict(saved_model.state_dict())
		else:
			self.load_state_dict(torch.load(save_path))

	def save_model(self, save_path, is_state_dict=False):
		if is_state_dict:
			torch.save(self.state_dict(), save_path)
		else:
			torch.save(self, save_path)

In [ ]:
torch.manual_seed(123)
def train_epoch(model, train_loader, optimizer):
  model.train()
  total = 0
  batch = 0
  total_loss = 0
  correct = 0
  for (input_batch, expected_out) in tqdm(train_loader, leave=False, desc="Training Batches"):
    optimizer.zero_grad()
    batch += 1
    output = model(input_batch.to(get_device())).to(get_device())
    loss = model.compute_Loss(output, expected_out)
    total += expected_out.size(dim=0)
    _, predicted = torch.max(output, -1)
    correct += (expected_out.to("cpu") == predicted.to("cpu")).cpu().numpy().sum()
    total_loss += loss.item()
    loss.backward()

    optimizer.step()
  print("Loss: " + str(total_loss/batch))
  print("Training Accuracy: " + str(correct/total))
  return total_loss/batch

In [ ]:
# Lambda to switch to GPU if available
get_device = lambda : "cuda:0" if torch.cuda.is_available() else "cpu"
device = get_device()

In [ ]:
torch.manual_seed(123)
def evaluation(model, val_loader, optimizer, test = False):
  model.eval()
  loss = 0
  correct = 0
  total = 0
  dataset = "Validation"
  if test:
      dataset = "Test"

  for (input_batch, expected_out) in tqdm(val_loader, leave=False, desc=dataset + " Batches"):
    output = model(input_batch.to(get_device())).to(get_device())
    total += expected_out.shape[0]
    _, predicted = torch.max(output, -1)
    correct += (expected_out.to("cpu") == predicted.to("cpu")).cpu().numpy().sum()
    loss += model.compute_Loss(output, expected_out)
  loss /= len(val_loader)
  print(dataset + " Loss: " + str(loss.item()))
  print(dataset + " Accuracy: " + str(correct/total))
  print()
  return loss.item()

In [ ]:
torch.manual_seed(123)
def train_and_evaluate(number_of_epochs, model, train_loader, val_loader, min_loss=0, lr=.001):
  optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=.01)
  loss_values = [[],[]]
  for epoch in trange(number_of_epochs, desc="Epochs"):
    cur_loss = train_epoch(model, train_loader, optimizer)
    loss_values[0].append(cur_loss)
    cur_loss_val = evaluation(model, val_loader, optimizer)
    loss_values[1].append(cur_loss_val)
    if cur_loss <= min_loss: return loss_values
  return loss_values

In [ ]:
torch.manual_seed(123)

vocab_size = len(vocab)
inp_dim = vocab_size # Don't change
h = 100 # Hidden dimension: feel free to change
out_dim = len(category_map) # Don't change
nhidden = 1 # Number of hidden layers: feel free to change
lr = 0.008 # Learning rate: feel free to change
model = FFNN(inp_dim, h, out_dim, vocab_size, nhidden)

nepochs = 15
trained_model = train_and_evaluate(nepochs, model, train_loader, val_loader, lr = lr)

Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Loss: 4.224335352579753
Training Accuracy: 0.00859106529209622


Validation Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Validation Loss: 4.105575084686279
Validation Accuracy: 0.05429864253393665



...

Validation Loss: 1.5284136533737183
Validation Accuracy: 0.6515837104072398



Training Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Loss: 0.33305450280507404
Training Accuracy: 0.8797250859106529


Validation Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Validation Loss: 1.5497758388519287
Validation Accuracy: 0.6561085972850679



In [ ]:
opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=.01)
evaluation(model, test_loader, opt, test = True)
pass

Test Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Test Loss: 1.2310601472854614
Test Accuracy: 0.6105769230769231



In [ ]:
embed_dim = 384
h_bert = 225
lr_bert = 0.006
nhidden_bert = 1
nepochs_bert = 30
model2 = FFNN(embed_dim, h_bert, out_dim, vocab_size, nhidden_bert)
trained_model_bert = train_and_evaluate(nepochs_bert, model2, train_loader_bert, val_loader_bert, lr = lr_bert)

Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Loss: 4.226936976114909
Training Accuracy: 0.018900343642611683


Validation Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Validation Loss: 4.040955066680908
Validation Accuracy: 0.26244343891402716



...

Validation Loss: 0.09400133043527603
Validation Accuracy: 0.9457013574660633



Training Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Loss: 0.02017293932537238
Training Accuracy: 0.9914089347079038


Validation Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Validation Loss: 0.09302251785993576
Validation Accuracy: 0.9457013574660633



In [ ]:
# test accuracy
opt_bert = optim.AdamW(model.parameters(), lr=lr_bert, weight_decay=.01)
evaluation(model2, test_loader_bert, opt_bert, test = True)
pass

Test Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Test Loss: 0.3770638108253479
Test Accuracy: 0.8798076923076923



In [ ]:
### Interactive BERT model
import random
print("Welcome! Enter a message for Cornbot or type \"quit\" to exit.")
while True:
    inp = input("You: ")
    if inp == 'quit':
        break
    vectorized_input = bert_sentences(inp)
    model_output = model2(torch.Tensor(vectorized_input))
    _, tag_int = torch.max(model_output, -1)
    tag_str = ''
    for key in category_map:
        if category_map[key] == tag_int:
            tag_str = key
    output = ''
    for intent in train["intents"]:
        if intent["tag"] == tag_str:
            index = random.randint(0, len(intent["responses"]) - 1)
            output = intent["responses"][index]
    print("Cornbot:", output)

Welcome! Enter a message for Cornbot or type "quit" to exit.
You: Tell me a joke?
Cornbot: Did you know Bill Nye graduated from Cornell?
You: When is morrison open?
Cornbot: Morrison is open from 7-4:30 and from 5-8:30
You: Where is duffield?
Cornbot: Duffield Hall is on the intersection between Feeney Way and Campus Road, right next to the engineering quad. It is a dark place.
You: When is add/drop for freshman fall?
Cornbot: Fall drop deadline is October 17th
You: What does trillium serve?
Cornbot: Trillium has has a burger station, quesadilla station, burrito station, wok station, salad station, pasta station, and a pizza station
You: Trillium?
Cornbot: Trillium has has a burger station, quesadilla station, burrito station, wok station, salad station, pasta station, and a pizza station
You: When is rose open?
Cornbot: Rose is open from 6-8
You: When is spring break?
Cornbot: Starts on Apr 1st, ends on the 10th.
You: What are your hobbies?
Cornbot: I enjoy providing quality informati

KeyboardInterrupt: ignored

In [ ]:
### Interactive bag of words model
print("Welcome! Enter a message for Cornbot or type \"quit\" to exit.")
while True:
    inp = input("You: ")
    if inp == 'quit':
        break
    padded_inp = pad_sents([inp.split()], PAD, max_len)
    vectorized_input = tokenize_sentences(padded_inp, vocab)
    model_output = model(torch.Tensor(vectorized_input))
    _, tag_int = torch.max(model_output, -1)
    tag_str = ''
    for key in category_map:
        if category_map[key] == tag_int:
            tag_str = key
    output = ''
    for intent in train["intents"]:
        if intent["tag"] == tag_str:
            index = random.randint(0, len(intent["responses"]) - 1)
            output = intent["responses"][index]
    print("Cornbot:", output)

Welcome! Enter a message for Cornbot or type "quit" to exit.
You: Hello
Cornbot: Hi, how are you?
You: How are you?
Cornbot: I'm doing well. How can I help you?
You: What is your favorite music?
Cornbot: I'm a big foodie, but K-BBQ is my favorite


KeyboardInterrupt: ignored